<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Разделение-и-маштабирование-данных" data-toc-modified-id="Разделение-и-маштабирование-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение и маштабирование данных</a></span></li><li><span><a href="#Определение-лучшей-модели-и-её-гиперпараметров" data-toc-modified-id="Определение-лучшей-модели-и-её-гиперпараметров-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Определение лучшей модели и её гиперпараметров</a></span></li><li><span><a href="#Проверка-качества-и-вменяемости-модели" data-toc-modified-id="Проверка-качества-и-вменяемости-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка качества и вменяемости модели</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоговый вывод</a></span></li></ul></div>

# Рекомендация тарифов

**Цель работы** — построить модель для оператора мобильной связи, планирующего перевести своих клиентов с архивных тарифов на один из новых. Модель будет определять, какой их двух актуальных тарифов стоит предложить пользователю.

В распоряжении имеются данные от оператора о поведении клиентов, которые уже перешли на эти тарифы.

**Ход работы**

Откроем файл с данными и изучим его. Предобработка этих данных была выполнена ранее, в данной работе осуществлять её не потребуется.

Разделим данные на обучающую и тестовую выборки, после чего маштабируем данные для более эффективной работы алгоритмов.

Далее рассмотрим разные модели и определим значения гиперпараметров, позволяющие получить наиболее качественную модель.

Выбрав таким образом модель с наилучшим показателем метрики, проверим её качество на тестовой выборке и её саму на вменяемость, после чего сделаем окончательный вывод о подходящей модели.
 
Таким образом, работа пройдёт в пять этапов:
 1. Обзор данных.
 2. Разделение и маштабирование данных.
 2. Определение лучшей модели и её гиперпараметров.
 3. Проверка качества и вменяемости модели.
 4. Итоговый вывод.

## Обзор данных

Взглянем на общую информацию о датасете.

In [5]:
# импорт библиотек
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [6]:
# чтение csv-файла и создание датафрейма
ub = pd.read_csv('data_users_behavior.csv')

# получение случайных 10 строк датасета
ub.sample(10, random_state=3)

,calls,minutes,messages,mb_used,is_ultra
2587,104.0,740.95,0.0,33702.04,1
3143,69.0,439.39,82.0,19315.86,1
2277,49.0,336.33,130.0,9777.89,1
1092,80.0,555.85,140.0,24412.57,1
471,89.0,528.61,53.0,16019.28,0
347,24.0,151.72,23.0,5957.73,0
2644,48.0,436.16,13.0,29072.01,0
838,61.0,452.89,27.0,10444.65,0
87,51.0,326.34,82.0,29084.42,1
562,35.0,229.53,66.0,17303.54,0


In [7]:
# получение общей информации о данных в таблице
ub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [8]:
# получение данных о характерных значениях таблицы
ub.describe().style.format('{:.2f}')

,calls,minutes,messages,mb_used,is_ultra
count,3214.00,3214.00,3214.00,3214.00,3214.00
mean,63.04,438.21,38.28,17207.67,0.31
std,33.24,234.57,36.15,7570.97,0.46
min,0.00,0.00,0.00,0.00,0.00
25%,40.00,274.58,9.00,12491.90,0.00
50%,62.00,430.60,30.00,16943.24,0.00
75%,82.00,571.93,57.00,21424.70,1.00
max,244.00,1632.06,224.00,49745.73,1.00


На первый взгляд с данными всё в порядке, исправим только тип данных у некоторых столбцов на `int`.

In [9]:
# изменение типов данных
ub[['calls', 'messages', 'is_ultra']] = (
    ub[['calls', 'messages', 'is_ultra']]
    .astype('int')
    )
# просмотр изменений
display(ub.sample(10, random_state=3))
ub.info()

,calls,minutes,messages,mb_used,is_ultra
2587,104,740.95,0,33702.04,1
3143,69,439.39,82,19315.86,1
2277,49,336.33,130,9777.89,1
1092,80,555.85,140,24412.57,1
471,89,528.61,53,16019.28,0
347,24,151.72,23,5957.73,0
2644,48,436.16,13,29072.01,0
838,61,452.89,27,10444.65,0
87,51,326.34,82,29084.42,1
562,35,229.53,66,17303.54,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int32  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int32  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int32  
dtypes: float64(2), int32(3)
memory usage: 88.0 KB


## Разделение и маштабирование данных

Для дальнейшей работы разделим датасет на на обучающую и тестовую выборки, также маштабируем данные для адекватных результатов всех используемых далее обучающих алгоритмов.

In [10]:
# разделение данных на выборки
features = ub.drop(['is_ultra'], axis=1)
target = ub['is_ultra']

features_train, features_test, target_train, target_test = (
    train_test_split(
        features,
        target,
        test_size=.3,
        random_state=4,
        stratify=target)
        )

# проверка размеров выборок
print(
    'Размеры обучающей выборки:',
    features_train.shape,
    target_train.shape
    )
print(
    'Размеры тестовой выборки:',
    features_test.shape,
    target_test.shape
    )

Размеры обучающей выборки: (2249, 4) (2249,)
Размеры тестовой выборки: (965, 4) (965,)


In [11]:
# маштабирование данных
scaler = RobustScaler()
scaler.fit(features_train, target_train)
features_train = scaler.transform(features_train)
features_test = scaler.transform(features_test)

## Определение лучшей модели и её гиперпараметров

Для получения подходящей модели будем выбирать между тремя алгоритмами обучения: логистической регрессией, "случайным лесом" и методом k-ближайших соседей.
У каждого алгоритма подберём наилучшие гиперпараметры с помощью метода `GridSearchCV`.

In [12]:
# определение лучших параметров и
# наивысшего accuracy для логистической регрессии
lr_clf = LogisticRegression(random_state=6)
lr_parametrs = {
    'C': range(1, 6),
    'class_weight': [None, 'balanced'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': range(10, 30)
    }
lr_grid = GridSearchCV(lr_clf, lr_parametrs, n_jobs=-1)
lr_grid.fit(features_train, target_train)

print(
    'Лучшие параметры для логистической регрессии:',
    lr_grid.best_params_
    )
print(f'Accuracy на обучающей выборке: {lr_grid.best_score_:.4f}')

Лучшие параметры для логистической регрессии: {'C': 1, 'class_weight': None, 'max_iter': 10, 'solver': 'newton-cg'}
Accuracy на обучающей выборке: 0.7394


In [13]:
%%time
# определение лучших параметров и
# наивысшего accuracy для случайного леса
rf_clf = RandomForestClassifier(random_state=7)
rf_parametrs = {
    'n_estimators': range(9, 40, 3),
    'min_samples_split': range(2, 6),
    'max_depth': range(5, 16),
    'class_weight': [None, 'balanced', 'balanced_subsample']
    }
rf_grid = GridSearchCV(rf_clf, rf_parametrs, n_jobs=-1)
rf_grid.fit(features_train, target_train)

print('Лучшие параметры для случайного леса:',
    rf_grid.best_params_
    )
print(f'Accuracy на обучающей выборке: {rf_grid.best_score_:.4f}')
print()
print('Время выполнения ячейки')

Лучшие параметры для случайного леса: {'class_weight': None, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 33}
Accuracy на обучающей выборке: 0.8052

Время выполнения ячейки
CPU times: total: 3.64 s
Wall time: 1min 16s


In [14]:
# определение лучших параметров и
# наивысшего accuracy для метода k-ближайших соседей
knn_clf = KNeighborsClassifier()
knn_parametrs = {
    'n_neighbors': range(5, 31),
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
    'algorithm': ['ball_tree', 'kd_tree', 'brute']
    }
knn_grid = GridSearchCV(knn_clf, knn_parametrs, n_jobs=-1)
knn_grid.fit(features_train, target_train)

print('Лучшие параметры для метода k-ближайших соседей:', 
    knn_grid.best_params_
    )
print(f'Accuracy на обучающей выборке: {knn_grid.best_score_:.4f}')

Лучшие параметры для метода k-ближайших соседей: {'algorithm': 'ball_tree', 'n_neighbors': 22, 'p': 2, 'weights': 'distance'}
Accuracy на обучающей выборке: 0.7986


Взглянем на графике, как изменяется значение `accuracy` при изменении одного из гиперпараметров лучшей в итоге модели — "случайного леса".

In [15]:
# получение графика
visual_rf_grid = GridSearchCV(
    rf_clf,
    {'max_depth': range(5, 16), 'min_samples_split': [4], 'n_estimators': [33]},
    n_jobs=-1
    )
visual_rf_grid.fit(features_train, target_train)
results = pd.DataFrame(visual_rf_grid.cv_results_)

px.line(
    results,
    x='param_max_depth',
    y='mean_test_score',
    markers=True,
    width=1000,
    height=500,
    title='Зависимость показателя метрики от гиперпараметра').update_layout(
        xaxis_title='max_depth',
        yaxis_title='Accuracy'
        )

Видно, что поначалу при увеличении глубины леса точность модели также ожидаемо увеличивается. Но после достижения оптимального значения дальнейшее увеличение параметра ведёт уже к ухудшению показателя метрики, что свидетельствует о переобучении.

**Промежуточные выводы**

По результатам подбора наилучшим вариантом в данном случае можно назвать модель, обученную с помощью "случайного леса" со следующими параметрами — `class_weight`: None, `max_depth`: 9, `min_samples_split`: 4, `n_estimators`: 33. Слабее всего себя показал метод логистической регресии. Значение метрики на валидационных выборках ни в одном из случаев сильно не снизилось, следовательно, переобучения не обнаружено.

## Проверка качества и вменяемости модели

Чтобы дать окончательный ответ о высоком уровне рекомендаций модели, проверим для неё значение различных метрик на тестовых данных и оценим модель на вменяемость. Для последнего сравним её `accuracy` со значением метрики константной модели на тестовых данных.  

In [16]:
# получение значений метрик на тестовых данных
predictions = rf_grid.predict(features_test)
best_clf_accuracy = accuracy_score(target_test, predictions)

print(f'Accuracy на тестовой выборке: {best_clf_accuracy:.3f}')
print(f'Precision на тестовой выборке: \
{precision_score(target_test, predictions):.3f}')
print(f'Recall на тестовой выборке: \
{recall_score(target_test, predictions):.3f}')
print(f'F1-мера на тестовой выборке: \
{f1_score(target_test, predictions):.3f}')
print(f'Количество TP-значений: \
{np.ravel(confusion_matrix(target_test, predictions))[3]}')
print(f'Количество FP-значений: \
{np.ravel(confusion_matrix(target_test, predictions))[1]}')
print(f'Количество TN-значений: \
{np.ravel(confusion_matrix(target_test, predictions))[0]}')
print(f'Количество FN-значений: \
{np.ravel(confusion_matrix(target_test, predictions))[2]}')

Accuracy на тестовой выборке: 0.818
Precision на тестовой выборке: 0.812
Recall на тестовой выборке: 0.527
F1-мера на тестовой выборке: 0.639
Количество TP-значений: 156
Количество FP-значений: 36
Количество TN-значений: 633
Количество FN-значений: 140


In [17]:
# определение адекватности модели
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_score = dummy_clf.score(features_train, target_train)
print(f'Accuracy константной модели на обучающих данных: {dummy_score:.3f}')

if best_clf_accuracy > dummy_score:
    print(
        'Accuracy итоговой модели больше accuracy аналогичной константной. Модель вменяема.'
        )
else:
    print(
        'Accuracy итоговой модели не выше accuracy аналогичной константной. Модель нельзя назвать вменяемой.'
        )

Accuracy константной модели на тестовых данных: 0.693
Accuracy итоговой модели больше accuracy аналогичной константной. Модель вменяема.


**Промежуточные выводы**

Можно утвержать, что полученная модель успешно прошла проверку на вменяемость. Её `accuracy` на тестовых данных равен примерно `0.82`, а значение метрики аналогичной константной модели `0.69`, то есть соотношение классов практически `7:3`. Следовательно, показатель метрики для утверждения об адекватности модели должен быть больше `0.7.` Значения остальных метрик выглядят также достаточными для эффективной работы модели.

## Итоговый вывод

Благодаря рассмотрению разных алгоритмов обучения и перебору их гиперпараметров с помощью метода, основанного на кросс-валидации, была получена модель с достаточно высокой долей правильных ответов. Данную модель можно смело предлагать оператору-заказчику для внедрения в систему рекомендаций новых тарифов пользователям.

Подобная система поможет компании перевести пользователей архивных тарифов на более выгодные для неё новые. За счёт верно подобранного тарифа количество переведённых таким образом клиентов, скорее всего, будет достаточно велико.